# **Sistema de recomendación de películas**

# **Pip & Requirements**

In [1]:
#pip install -r /workspaces/KNN-project/requirements.txt

In [2]:
#pip install pyarrow

In [3]:
#pip install plotly


# Globals

In [1]:
tfidf = True
n_recommended = 5
limit_output = 10

# **All Imports**

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import json
import seaborn as sns
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import os
import plotly.express as px
from pickle import dump

## **Step 1: Import Data**

In [3]:
movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
#Hacer merge con pandas, prueba

In [11]:
"""# Leer los dataframes
movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

# Fusionar los dataframes en base a la columna 'id'
merged_df = pd.merge(movies, credits, left_on='id', right_on='movie_id')

# Mostrar las primeras filas del dataframe fusionado
print(merged_df.head())"""


'# Leer los dataframes\nmovies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")\ncredits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")\n\n# Fusionar los dataframes en base a la columna \'id\'\nmerged_df = pd.merge(movies, credits, left_on=\'id\', right_on=\'movie_id\')\n\n# Mostrar las primeras filas del dataframe fusionado\nprint(merged_df.head())'

### Step 2 Conexion a SQL

In [5]:
conn = sqlite3.connect("../data/movies_database.db")
movies.to_sql("movies",conn, if_exists="replace")
credits.to_sql("credits", conn, if_exists="replace")
cursor = conn.cursor()

In [6]:
cursor.execute("SELECT * FROM movies")

In [1]:
#for row in cursor.fetchall(): #error con el repository
    #print(row)

In [8]:
join = "SELECT * FROM movies INNER JOIN credits on movies.title = credits.title;"

In [5]:
#cursor.execute(join)
#for row in cursor.fetchmany(5):  #errror en el commit
    #print(row)

In [10]:
join_data = pd.read_sql_query(join,conn)
conn.close
join_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,status,tagline,title,vote_average,vote_count,index,movie_id,title,cast,crew
0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,Released,Enter the World of Pandora.,Avatar,7.2,11800,0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,Released,A Plan No One Escapes,Spectre,6.3,4466,2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,Released,"Lost in our world, found in another.",John Carter,6.1,2124,4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [12]:
print(credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB
None


In [13]:
print(join_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4809 non-null   int64  
 1   budget                4809 non-null   int64  
 2   genres                4809 non-null   object 
 3   homepage              1713 non-null   object 
 4   id                    4809 non-null   int64  
 5   keywords              4809 non-null   object 
 6   original_language     4809 non-null   object 
 7   original_title        4809 non-null   object 
 8   overview              4806 non-null   object 
 9   popularity            4809 non-null   float64
 10  production_companies  4809 non-null   object 
 11  production_countries  4809 non-null   object 
 12  release_date          4808 non-null   object 
 13  revenue               4809 non-null   int64  
 14  runtime               4807 non-null   float64
 15  spoken_languages     

In [14]:
print(join_data.columns)

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'index', 'movie_id', 'title', 'cast',
       'crew'],
      dtype='object')


In [15]:
join_data.drop("index", axis=1, inplace=True)

In [16]:
print(join_data.genres[0])

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


### Step 3 Preprocessing Genres

In [17]:
genres = []
for i in range(len(join_data.genres)):
    genres_list = []
    for d in json.loads(join_data.genres[i]):
        #print(d['name'])
        genres_list.append(d['name'])
    #print('\n')
    genres.append(f"[{','.join(genres_list)}]")
    #print(f"[{','.join(genres_list)}]")
join_data.genres = genres

In [18]:
join_data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[Action,Adventure,Fantasy,Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure,Fantasy,Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action,Adventure,Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action,Crime,Drama,Thriller]",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action,Adventure,Science Fiction]",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


### Step 4 Preprocesiamiento Keywords

In [19]:
keywords = []
for i in range(len(join_data.keywords)):
    keywords_list = []
    for d in json.loads(join_data.keywords[i]):
        keywords_list.append(d['name'])
    keywords.append(f"[{','.join(keywords_list)}]")
join_data.keywords = keywords

In [20]:
join_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

#### Step 5 preprocesiamiento cast

In [21]:
cast = []
for i in range(len(join_data.cast)):
    cast_list = []
    for d in json.loads(join_data.cast[i]):
        cast_list.append(d['name'])
    cast.append(f"[{','.join(cast_list[0:3])}]")
join_data.cast = cast

In [22]:
join_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

### Step 6 preprocesiamiento crew

In [23]:
crew = []
for i in range(len(join_data.crew)):
    crew_list = []
    for d in json.loads(join_data.crew[i]):
        if d['job'].lower().strip() == 'director':
            crew_list.append(d['name'])
    crew.append(f"[{','.join(crew_list[0:3])}]")
join_data.crew = crew

### Step 7 preprocesiamiento overview

In [24]:
join_data.overview[0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [25]:
overview = []
for o in join_data.overview:
    overview.append(f"[{o}]")
join_data.overview = overview

In [26]:
train_data= pd.DataFrame()

In [27]:
train_data["tags"] = (join_data["overview"] + join_data["genres"] + join_data["keywords"] + join_data["cast"] + join_data["crew"]).str.replace(","," ").str.replace("["," ").str.replace("]"," ")

In [28]:
train_data["title"] = join_data["title"].iloc[:,0]
train_data

,tags,title
0,In the 22nd century a paraplegic Marine is d...,Avatar
1,Captain Barbossa long believed to be dead h...,Pirates of the Caribbean: At World's End
2,A cryptic message from Bond’s past sends him ...,Spectre
3,Following the death of District Attorney Harv...,The Dark Knight Rises
4,John Carter is a war-weary former military c...,John Carter
...,...,...
4804,El Mariachi just wants to play his guitar and...,El Mariachi
4805,A newlywed couple's honeymoon is upended by t...,Newlyweds
4806,"""Signed Sealed Delivered"" introduces a dedi...","Signed, Sealed, Delivered"
4807,When ambitious New York attorney Sam is sent ...,Shanghai Calling


In [29]:
train_data

,tags,title
0,In the 22nd century a paraplegic Marine is d...,Avatar
1,Captain Barbossa long believed to be dead h...,Pirates of the Caribbean: At World's End
2,A cryptic message from Bond’s past sends him ...,Spectre
3,Following the death of District Attorney Harv...,The Dark Knight Rises
4,John Carter is a war-weary former military c...,John Carter
...,...,...
4804,El Mariachi just wants to play his guitar and...,El Mariachi
4805,A newlywed couple's honeymoon is upended by t...,Newlyweds
4806,"""Signed Sealed Delivered"" introduces a dedi...","Signed, Sealed, Delivered"
4807,When ambitious New York attorney Sam is sent ...,Shanghai Calling


In [30]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tags    4809 non-null   object
 1   title   4809 non-null   object
dtypes: object(2)
memory usage: 75.3+ KB


In [31]:
train_data.describe()

,tags,title
count,4809,4809
unique,4809,4800
top,In the 22nd century a paraplegic Marine is d...,The Host
freq,1,4


In [32]:
train_data.to_csv("../data/processed/clean_data.csv", index = False)


### Step 8 Visualizing 

In [73]:
import plotly.express as px

fig = px.scatter_3d(join_data, x="budget", y="popularity", z="revenue", width=1000, height=500,
                    size=join_data["budget"].abs(), color="revenue", color_continuous_scale="Viridis")
camera = dict(
    up=dict(x=1, y=3.5, z=0),
    eye=dict(x=2, y=0, z=0)
)

fig.update_layout(scene_camera=camera)
fig.show()


In [33]:
if tfidf == True:
  vectorizer = TfidfVectorizer()
else:
  vectorizer = CountVectorizer()
tdm = vectorizer.fit_transform(train_data.tags)

In [34]:
train_data['tags'].apply(lambda x: len(x.split())).sum


<bound method Series.sum of 0        72
1        70
2        65
3       107
4        94
       ... 
4804     83
4805     23
4806     96
4807     73
4808     80
Name: tags, Length: 4809, dtype: int64>

In [35]:
tdm.toarray().shape

(4809, 28796)

### Step 9 K-nearest Model

In [36]:
model = NearestNeighbors(n_neighbors = n_recommended, metric = 'cosine')
model.fit(tdm)


NearestNeighbors(metric='cosine')

In [44]:
"""def recommend(movie, library):
    movie_index = library[library['title'].str.lower().str.strip() == movie.lower().strip()].index[0]
    distances = cosine_similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True , key = lambda x: x[1])[1:6]

    for i in movie_list:
        print(library.iloc[i[0]].title)"""

#### Step 10 Recomend the movies Data + Model

In [37]:
def recommend(movie, library, tdm):
    movie_index = library[library['title'].str.lower().str.strip() == movie.lower().strip()].index[0]
    distances, indices = model.kneighbors(tdm[movie_index])
    similar_movies = [(library["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

In [38]:
all_recs = []
if limit_output is not None:
  if limit_output > len(train_data["title"]):
    limit_output = len(train_data["title"])
else:
    limit_output = len(train_data["title"])

for i in range(limit_output):
  recs = []
  rec = recommend(train_data.iloc[i]["title"], train_data, tdm)
  for movie, _ in rec:
    recs.append(movie)
  all_recs.append(recs)

  print(recs)

"""max_len = max(len(sublist) for sublist in recs)
padded_lists = [sublist + [None] * (max_len - len(sublist)) for sublist in recs]

train_data["recs"] = padded_lists"""

['Aliens', 'Alien³', 'Moonraker', 'Mission to Mars']
["Pirates of the Caribbean: Dead Man's Chest", 'Pirates of the Caribbean: The Curse of the Black Pearl', 'Pirates of the Caribbean: On Stranger Tides', 'The Pirates! In an Adventure with Scientists!']
['Skyfall', 'Never Say Never Again', 'From Russia with Love', 'Quantum of Solace']
['The Dark Knight', 'Batman Returns', 'Batman Begins', 'Batman']
['Mission to Mars', 'The Martian', 'Ghosts of Mars', 'Avatar']
['Spider-Man', 'Spider-Man 2', 'Arachnophobia', 'The Amazing Spider-Man']
['Into the Woods', 'Enchanted', 'Ella Enchanted', 'Shrek 2']
['The Avengers', 'Iron Man 2', 'Captain America: Civil War', 'Iron Man']
['Harry Potter and the Order of the Phoenix', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Chamber of Secrets', "Harry Potter and the Philosopher's Stone"]
['Man of Steel', 'Batman Begins', 'The Dark Knight', 'Batman Returns']


'max_len = max(len(sublist) for sublist in recs)\npadded_lists = [sublist + [None] * (max_len - len(sublist)) for sublist in recs]\n\ntrain_data["recs"] = padded_lists'

In [42]:
recs

['Man of Steel', 'Batman Begins', 'The Dark Knight', 'Batman Returns']

In [50]:
def get_movie_recommendations(movie_title):
    movie_index = train_data[train_data["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tdm[movie_index])
    similar_movies = [(train_data["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "Iron Man 2"
recommendations = get_movie_recommendations(input_movie)

all_recs = []
for movie, _ in recommendations:
    all_recs.append(movie)

print("Film recommendations for '{}':".format(input_movie))
for movie in all_recs:
    print("- Film: {}".format(movie))


Film recommendations for 'Iron Man 2':
- Film: Iron Man 3
- Film: Iron Man
- Film: Avengers: Age of Ultron
- Film: The Avengers


### Step 11 save up 

In [ ]:
from pickle import dump

dump(model, open("../models/knn_neighbors-6_algorithm-brute_metric-cosine.sav", "wb"))